In [98]:
import pandas as pd
import re as re

pd.set_option('display.max_columns', 500)

In [99]:
# Take in Excel

itemized = pd.read_excel("C:/Users/danie/OneDrive/Projects/fedex_reconciliation/Invoice_data.xlsx", header=2, sheet_name=0)
invoice_data = pd.read_excel("C:/Users/danie/OneDrive/Projects/fedex_reconciliation/Invoice_data.xlsx", sheet_name=1)
qbo = pd.read_excel("C:/Users/danie/OneDrive/Projects/fedex_reconciliation/QBO_customers(1).xlsx")
amt = pd.read_excel("C:/Users/danie/OneDrive/Projects/fedex_reconciliation/Exensiv.xlsx", sheet_name='AMT')
gp_acoustics = pd.read_excel("C:/Users/danie/OneDrive/Projects/fedex_reconciliation/Exensiv.xlsx", sheet_name='GPAcoustics')

In [100]:
invoice_data['Source'] = 'Invoice Data'
qbo['Source'] = 'QBO'
amt['Source'] = 'Extensiv'
gp_acoustics['Source'] = 'Extensiv'

In [101]:
df = pd.concat(objs=[invoice_data,qbo, amt, gp_acoustics],join='outer')

In [102]:
amt = df[(df['CustomerIdentifier.Name'] == 'AMT') &  (df['Source'] == 'Extensiv')]
gp_acoustics = df[(df['CustomerIdentifier.Name'] == 'GP Acoustics') &  (df['Source'] == 'Extensiv')]
qbo = df[df['Source'] == 'QBO']
invoice_data = df[df['Source'] == 'Invoice Data']

In [103]:
df['Source'].unique()

array(['Invoice Data', 'QBO', 'Extensiv'], dtype=object)

In [104]:
amt = amt.dropna(axis=1, how='all')
gp_acoustics = gp_acoustics.dropna(axis=1, how='all')
qbo = qbo.dropna(axis=1, how='all')
invoice_data = invoice_data.dropna(axis=1, how='all')

In [105]:
def compare_qbo(qbo: pd.DataFrame, invoice_data: pd.DataFrame) -> pd.DataFrame:
    """
    Function: Compares FedEx invoice with QuickBooks via keys 'Customer PO #' and 'Display_Name'
    Input: Original QuickBooks and FedEx Invoice file
    Output: Pandas DataFrame with values not found in QuickBooks
    """

    # ? is 'Display_Name' the only key to compare against?

    qbo_found = pd.merge(
        qbo, invoice_data, right_on="Customer PO #", left_on="Display_Name", how="inner", suffixes=['_qbo','_invoice_data']
    )

    lst = set()
    for i in invoice_data["Customer PO #"]:
        if i not in list(qbo_found["Display_Name"].unique()):
            lst.add(i)

    qbo_not_found = pd.DataFrame()
    qbo_not_found["Customer PO #"] = pd.DataFrame(lst)
    qbo_not_found = qbo_not_found.merge(
        invoice_data,
        on="Customer PO #",
        how="left",
    )
    return qbo_found, qbo_not_found

In [106]:
qbo_found, qbo_not_found = compare_qbo(qbo, invoice_data)

In [107]:
qbo_found[~qbo_found['Display_Name'].duplicated()]

,Source_qbo,Id,SyncToken,Create_Time,Last_Updated_Time,Given_Name,Display_Name,Fully_Qualified_Name,Company_Name,Print_OnCheck_Name,PrimaryPhone_FreeFormNumber,Mobile_FreeFormNumber,Primary_Email_Address,Web_Address_URI,DefaultTaxCodeRef_Value,Taxable,Bill_Address_Id,Bill_Address_Line1,Bill_Address_Line2,Bill_Address_City,Bill_Address_Country,Bill_Address_Country_SubDivision_Code,Bill_Address_Postal_Code,Ship_Address_Id,Ship_Address_Line1,Ship_Address_Line2,Ship_Address_City,Ship_Address_Country,Ship_Address_Country_SubDivision_Code,Ship_Address_Postal_Code,Notes,Job,Bill_With_Parent,ParentRef_Value,Level,SalesTermRef_Value,PaymentMethodRef_Value,Balance,Balance_With_Jobs,CurrencyRef_Value,CurrencyRef_Name,Shipper #,Invoice #,Invoice Date,Invoice Amount,Tracking #,Ship Date,Delivery Date,Delivery Time,Service Level,Zone,Reference,Department,Bill Option,Actual Weight,Bill Weight,Shipper Name,Shipper Company,Shipper Address,Shipper City,Shipper State,Shipper Postal Code,Shipper Country,Receiver Name,Receiver Company,Receiver Address,Receiver City,Receiver State,Receiver Postal Code,Receiver Country,Customer PO #,Declared Value,Customs Value,Shipper Account,Reference 2,Length,Width,Height,Control #,Published Amount,Discounted Amount,Residential,DAS,Fuel,Saturday Delivery,Add'd Handling,Misc. Charges,Client,Coding,Total Charges,Adjusted Amount,Audited Amount,Service Packaging,Source_invoice_data
0,QBO,1453.0,0.0,2024-09-26 15:31:33,2024-09-26 15:31:33,NaN,ARRCO-22724-00024,Advantage Sales & Marketing:ARRCO-22724-00024,NaN,ARRCO-22724-00024,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,1.0,1.0,NaN,NaN,0.0,0.0,USD,United States Dollar,693070511.0,867618864.0,2024-11-08,25739.62,4.190644e+11,2024-10-29,2024-10-31,09:43:00,Ground,5.0,Nautical/Arrived/Splenda,NaN,Prepaid,5.1,8.0,Jeff Flassig,Advantage Solutions,16000 W 116th st,Lenexa,KS,66219.0,US,MIA BENGE,ORANGETHEORY FITNESS - KYLE,5940 KYLE PKWY,KYLE,TX,786402482.0,US,ARRCO-22724-00024,0.0,0.0,693070511.0,NaN,20.0,13.0,10.0,ADVA4456,17.90,-13.23,0.00,0.00,0.40,0.0,0.0,0.23,Nautical,59062/587120,5.30,0.0,5.30,Customer Packaging,Invoice Data
724,QBO,2314.0,0.0,2024-11-25 15:06:57,2024-11-25 15:06:57,NaN,ARRCO-26424-00035,Advantage Sales & Marketing:ARRCO-26424-00035,NaN,ARRCO-26424-00035,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,1.0,1.0,NaN,NaN,0.0,0.0,USD,United States Dollar,693070511.0,868389948.0,2024-11-15,14296.89,4.190644e+11,2024-11-07,2024-11-08,10:13:00,Ground,2.0,Nautical/Arrived/Horizon,NaN,Prepaid,31.1,32.0,Jeff Flassig,Advantage Solutions,16000 W 116th st,Lenexa,KS,66219.0,US,LINDA GOTT,WEST HEIGHTS UNITED METHODIST PRESC,745 N WESTLINK AVE,WICHITA,KS,672124404.0,US,ARRCO-26424-00035,0.0,0.0,693070511.0,NaN,20.0,13.0,12.0,ADVA4465,21.62,-16.19,0.00,0.00,0.44,0.0,0.0,0.00,Nautical,59062/587120,5.87,0.0,5.87,Customer Packaging,Invoice Data
1674,QBO,2452.0,0.0,2024-12-04 14:07:26,2024-12-04 14:07:26,NaN,ARRCO-22724-00027,Arrived Co:ARRCO-22724-00027,NaN,ARRCO-22724-00027,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,408.0,1.0,NaN,NaN,0.0,0.0,USD,United States Dollar,693070511.0,868389946.0,2024-11-15,27353.46,4.163984e+11,2024-11-04,2024-11-06,12:03:00,Ground,4.0,Nautical/Arrived/Splenda,NaN,Prepaid,2.1,5.0,Jeff Flassig,Advantage Solutions,16000 W 116th st,lenexa,KS,66219.0,US,RENEE DOOLEY,RESILIENT HEALTH AND PERFORMANCE,204 WARD CIR,BRENTWOOD,TN,370277551.0,US,ARRCO-22724-00027,0.0,0.0,693070511.0,NaN,14.0,12.0,10.0,ADVA4465,15.33,-10.84,0.00,0.00,0.36,0.0,0.0,0.00,Nautical,59062/587120,4.85,0.0,4.85,Customer Packaging,Invoice Data
1798,QBO,2453.0,0.0,2024-12-04 14:07:58,2024-12-04 14:07:58,NaN,ARRCO-22724-00028,Arrived Co:ARRCO-22724-00028,NaN,ARRCO-22724-00028,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,408.0,1.0,NaN,NaN,0.0,0.0,USD,United States Dollar,693070511.0,8683

In [108]:
def add_pattern_column(invoice_data: pd.DataFrame) -> pd.DataFrame:
    """
    Function: Adds a column to the FedEx Invoice DataFrame with the RegEx pattern symbolizing 'Customer PO #'
    Input: FedEx Invoice DataFrame
    Output: DataFrame with added 'Pattern' column
    """

    def reg_tokenizer(value):

        with_letters = re.sub(r"[a-zA-Z]+", r"\\w+", str(value))
        with_numbers = re.sub(r"\d+", r"\\d+", with_letters)
        with_spaces = re.sub(r"\s+", r"\\s+", with_numbers)

        final = re.compile(with_spaces)

        return final

    token_lst = []

    for i in invoice_data["Reference"]:
        token_lst.append(reg_tokenizer(i))

    invoice_data["Pattern"] = token_lst

    return invoice_data

In [109]:
invoice_data = add_pattern_column(invoice_data)

In [112]:
def find_extensiv_reference_columns(extensiv_table: pd.DataFrame, invoice_data_w_patterns: pd.DataFrame) -> dict:  # fmt: skip
    """
    Function: Finds all of the columns in the Extensiv table that match each 'Reference' in FedEx Invoice not in QBO
    Input: Extensiv DataFrame, FedEx Invoice DataFrame w/ added 'Pattern' column
    Ouput: Dictionary {'match_lst': list of Extensiv columns that match 'Reference' pattern,
                       'Total Charges': Charges associated with that 'Reference' in FedEx Invoice,
                       'Tracking #': Tracking number associated with that 'Reference' in FedEx Invoice}
    Notes: May not need Total Charges and Tracking # in the end
    """

    def find_col_match(extensiv_table: pd.DataFrame, ref_pattern: pd.Series) -> list:
        """
        Function: Subfunction to iterate through each of the patterns in FedEx Invoice
        Input: Extensiv DataFrame, FedEx Reference patterns as a Series in a for loop
        Ouput: List of columns that match given Reference pattern
        """
        col_lst = set()

        for col in extensiv_table.columns:

            for value in extensiv_table[col]:

                if isinstance(value, float) and value.is_integer():
                    
                    value = int(value)

                if re.fullmatch(ref_pattern, str(value)):

                    col_lst.add(col)
                    break

                else:
                    break

        if len(col_lst) != 0:
            return col_lst
        else:
            return None

    match_dct = dict()
    suffix = 0

    for i, v in enumerate(invoice_data_w_patterns["Reference"]):

        if i != 0 and v == invoice_data_w_patterns["Reference"][i - 1]:

            suffix += 1
            v = f"{v}-s{suffix}"

        elif i != 0 and v != invoice_data_w_patterns["Reference"][i - 1]:
            suffix = 0
        else:
            continue

        match_lst = find_col_match(
            extensiv_table, invoice_data_w_patterns["Pattern"][i]
        )

        if match_lst is not None and not pd.isna(v):

            match_dct[v] = {
                "match_lst": match_lst,
                "Total Charges": invoice_data_w_patterns["Total Charges"][i],
                "Tracking #": invoice_data_w_patterns["Tracking #"][i],
            }

    return match_dct

In [113]:
gp_reference_columns = find_extensiv_reference_columns(gp_acoustics, invoice_data)

In [114]:
gp_reference_columns

{10554: {'match_lst': {'AsnCandidate',
   'Column1',
   'CreatedByIdentifier.Id',
   'Custom.TotalResults',
   'CustomerIdentifier.Id',
   'Index',
   'LastModifiedByIdentifier.Id',
   'LoadedState',
   'NumUnits1',
   'OrderId',
   'Packages.CartonId',
   'Packages.Cod',
   'Packages.CodAmount',
   'Packages.Height',
   'Packages.InsuredAmount',
   'Packages.Oversize',
   'Packages.PackageId',
   'Packages.PackageTypeId',
   'Packages.Ucc128',
   'Packages.Width',
   'ParcelLabelType',
   'ReferenceNum',
   'RouteCandidate',
   'RoutingInfo.BillOfLading',
   'RoutingInfo.Mode',
   'RoutingInfo.TrackingNumber',
   'ShipTo.AddressStatus',
   'ShipTo.ContactId',
   'ShipTo.Zip',
   'Status',
   'TransactionEntryType',
   'WarehouseTransactionSourceType'},
  'Total Charges': np.float64(6.54),
  'Tracking #': np.float64(280881245477.0)},
 10555: {'match_lst': {'AsnCandidate',
   'Column1',
   'CreatedByIdentifier.Id',
   'Custom.TotalResults',
   'CustomerIdentifier.Id',
   'Index',
   'La

In [122]:
def find_value_match(extensiv_table: pd.DataFrame, reference_matches: dict) -> list:

    match_lst = list()

    for reference in reference_matches:

        matches = reference_matches[reference]["match_lst"]
        total_charges = reference_matches[reference]["Total Charges"]
        tracking_number = reference_matches[reference]["Tracking #"]

        for col in extensiv_table[list(matches)]:

            for i, val in enumerate(extensiv_table[col]):

                base_reference = re.sub(r"-s\d+$", "", str(reference))

                if val == reference or val == base_reference:

                    match_entry = {
                        "Reference": base_reference,
                        "Name": extensiv_table["CustomerIdentifier.Name"][i],
                        "Column": col,
                        "Total Charges": total_charges,
                        "Tracking #": tracking_number,
                    }

                    if match_entry not in match_lst:
                        match_lst.append(match_entry)

    if not match_lst:
        print(f"No Matches")

    else:
        return pd.DataFrame(match_lst)

In [123]:
gp_reference_matches = find_value_match(gp_acoustics,gp_reference_columns)

In [124]:
gp_reference_matches

,Reference,Name,Column,Total Charges,Tracking #
0,10554,GP Acoustics,OrderId,6.54,2.808812e+11
1,10555,GP Acoustics,OrderId,5.85,2.808814e+11
2,10556,GP Acoustics,OrderId,4.92,2.808815e+11
3,10557,GP Acoustics,OrderId,5.55,2.808818e+11
4,10558,GP Acoustics,OrderId,4.92,2.808817e+11
5,10559,GP Acoustics,OrderId,12.10,2.808821e+11
6,10560,GP Acoustics,OrderId,8.21,2.808819e+11
7,10561,GP Acoustics,OrderId,7.47,2.808816e+11
8,10562,GP Acoustics,OrderId,10.63,2.808815e+11
9,10563,GP Acoustics,OrderId,5.56,2.808817e+11
